In [ ]:
#############################################################################
# zlib License
#
# (C) 2023 Murtaza Safdari <musafdar@cern.ch>, Jongho Lee <jongho.lee@cern.ch>
#
# This software is provided 'as-is', without any express or implied
# warranty.  In no event will the authors be held liable for any damages
# arising from the use of this software.
#
# Permission is granted to anyone to use this software for any purpose,
# including commercial applications, and to alter it and redistribute it
# freely, subject to the following restrictions:
#
# 1. The origin of this software must not be misrepresented; you must not
#    claim that you wrote the original software. If you use this software
#    in a product, an acknowledgment in the product documentation would be
#    appreciated but is not required.
# 2. Altered source versions must be plainly marked as such, and must not be
#    misrepresented as being the original software.
# 3. This notice may not be removed or altered from any source distribution.
#############################################################################

In [ ]:
from beamtest_analysis_helper import etroc2_analysis_helper
import datetime
from pathlib import Path
import pandas as pd
import numpy as np
from glob import glob
from natsort import natsorted

In [ ]:
# !!!!!!!!!!!!
# It is very important to correctly set the chip name, this value is stored with the data
chip_names = ["ET2_W36_IP7_13_HV210V_offset20","ET2_EPIR_1_1_HV210V_offset20", "ET2_CNM_1_3_HV210V_offset20"]
chip_fignames = chip_names
chip_figtitles = ["ETROC2 WB W36 IP7-13 HV210V OS:20","ETROC2 BB EPIR 1-1 HV210V OS:20", "(Trigger) ETROC2 BB CNM 1-3 HV210V OS:20"]

chip_labels= ["1","0","3"]

today = datetime.date.today().isoformat()
fig_outdir = Path('../../ETROC-figures')
fig_outdir = fig_outdir / (today + '_Array_Test_Results')
fig_outdir.mkdir(exist_ok=True)
fig_path = str(fig_outdir)

# path_pattern = f"*2023-09-21_Array_Test_Results/SelfTrigger_bottom_Readout_topbottom_1"
path_pattern = f"./SelfTrigger_ET2_CNM_BATCH_1_3_Readout_ET2_EPIR_BATCH1_1_ET2_W36_IP7_13_ET2_CNM_BATCH1_3_loop_*.pqt"

helper = etroc2_analysis_helper(chip_names=chip_names, chip_figtitles=chip_figtitles, chip_labels=chip_labels)

In [ ]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt

fig = plt.figure(dpi=50, figsize=(5,5))
gs = fig.add_gridspec(1,1)

ax0 = fig.add_subplot(gs[0,0])
ax0.plot([1, 0], [1, 0])
plt.show()

In [ ]:
def pixelID_filter(group):
    return any((group['board'] == 1) & (group['row'] == 15) & (group['col'] == 6)) & \
            any((group['board'] == 0) & (group['row'] == 2) & (group['col'] == 6)) & \
            any((group['board'] == 3) & (group['row'] == 2) & (group['col'] == 5))

In [ ]:
files = glob(path_pattern)
files = natsorted(files)

dataframes = []

for ifile in files:
    tmp_df = pd.read_parquet(ifile)

    # Group the DataFrame by 'evt' and count unique 'board' values in each group
    unique_board_counts = tmp_df.groupby('evt')['board'].nunique()

    ## event has two unique board ID
    event_numbers_with_three_unique_boards = unique_board_counts[unique_board_counts == 3].index
    subset_df = tmp_df[tmp_df['evt'].isin(event_numbers_with_three_unique_boards)]
    subset_df.reset_index(inplace=True, drop=True)

    del tmp_df
    if subset_df.empty: continue

    ## event has one hit from each board
    event_board_counts = subset_df.groupby(['evt', 'board']).size().unstack(fill_value=0)
    selected_event_numbers = event_board_counts[(event_board_counts[0] == 1) & (event_board_counts[1] == 1) & (event_board_counts[3] == 1)].index
    selected_subset_df = subset_df[subset_df['evt'].isin(selected_event_numbers)]
    selected_subset_df.reset_index(inplace=True, drop=True)
    dataframes.append(selected_subset_df)

df = pd.concat(dataframes)

In [ ]:
df.info()

In [ ]:
tmp_group = df.groupby('evt')
filtered_group = tmp_group.filter(pixelID_filter)
filtered_group.reset_index(inplace=True, drop=True)
filtered_group

In [ ]:
h_selection = helper.return_hist(filtered_group, chip_names, chip_labels)

In [ ]:
helper.make_pix_inclusive_plots(h_selection, chip_names[0], chip_fignames[0], chip_figtitles[0], fig_path, save=False, show=True, tag="Pixel (15, 6)", title_tag=", Pixel (15, 6)")

In [ ]:
helper.make_pix_inclusive_plots(h_selection, chip_names[1], chip_fignames[1], chip_figtitles[1], fig_path, save=False, show=True, tag="Pixel (2, 6)", title_tag=", Pixel (2, 6)")

In [ ]:
helper.make_pix_inclusive_plots(h_selection, chip_names[2], chip_fignames[2], chip_figtitles[2], fig_path, save=False, show=True, tag="Pixel (2, 5)", title_tag=", Pixel (2, 5)")

In [ ]:
del h_selection

In [ ]:
cal_cut = [150, 250] # min, max
toa_cut = [100, 500] # min, max
tot_cut = [ 50, 300] # min, max

## Boundary values are included by default:
selected_df = filtered_group[filtered_group['cal'].between(cal_cut[0], cal_cut[1]) &
                            filtered_group['toa'].between(toa_cut[0], toa_cut[1]) &
                            filtered_group['tot'].between(tot_cut[0], tot_cut[1])]

selected_df.reset_index(inplace=True, drop=True)

In [ ]:
h_after_tdc = helper.return_hist(selected_df, chip_names, chip_labels)

In [ ]:
helper.make_pix_inclusive_plots(h_after_tdc, chip_names[0], chip_fignames[0], chip_figtitles[0], fig_path, save=False, show=True, tag="Pixel (15, 6)", title_tag=", Pixel (15, 6) after TDC cut")

In [ ]:
helper.make_pix_inclusive_plots(h_after_tdc, chip_names[1], chip_fignames[1], chip_figtitles[1], fig_path, save=False, show=True, tag="Pixel (2, 6)", title_tag=", Pixel (2, 6) after TDC cut")

In [ ]:
helper.make_pix_inclusive_plots(h_after_tdc, chip_names[2], chip_fignames[2], chip_figtitles[2], fig_path, save=False, show=True, tag="Pixel (2, 5)", title_tag=", Pixel (2, 5) after TDC cut")

In [ ]:
del h_after_tdc